In [1]:
from visualizer import calibration
#TODO: import from waveorder
import visual 

from PyQt5 import QtCore

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [81]:
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_2_1a' # 5 Movement Across FOV
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_2_2a' # 4 Brighter signal but not much movement
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_2_3a' # 3 Egg in 3D FOV. Not much movement
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_2_4a' # 3 Movement but very low signal
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_2_5a' # 5 Bright and two half worms (L3) moving in FOV
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_3_3a_WL' # 3 Slow WL imaging
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_OH1_L1_6a' # 5 Birght and some movement of two worms

# main_folder = r'E:\Ed\Data\20220413_celegans_100FPS\20220413_M25_OH1_L1_1a' # 5 Accidental WL and Fluorescence
# main_folder = r'E:\Ed\Data\20220413_celegans_100FPS\20220413_M25_OH1_L1_2a' # 4 some wiggle with set of neurons
# main_folder = r'E:\Ed\Data\20220413_celegans_100FPS\20220413_M25_OH1_L1_3a' #  4 lots of 3D wiggle
# main_folder = r'E:\Ed\Data\20220413_celegans_100FPS\20220413_M25_OH1_L1_4a' # 5 Accidental WL and Fluorescence
# main_folder = r'E:\Ed\Data\20220413_celegans_100FPS\20220413_M25_OH1_L1_5a' # 5 Moevement of L4's in 3D



### ZW495 
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_1_1a' # 3 Not dynamic good signal/
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_1_2a' # 3 Not much movement. two side by side
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_1_3a' # 5 fast movement in 3D
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_1_4a' # 4 some wiggle 
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_2_1a' # deleted no data
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_2_2a' # 5 amazing curled up worm in 3D
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_2_3a' # 3 good signal not much movement
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_2_4a' # 2 they come in FOV at the end
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_2_5a' # 4 multiple worms in/out FOV. Good 3D
# main_folder = r'D:\Ed\Data\20220413_M25_Celegans_100FPS\20220413_M25_zw495_L1_2_6a' # 5 head and some slow wiggle. good 3D?


#50FPS
main_folder = r'D:\Ed\Data\20220413_M25_Celegans_50FPS\20220413_M25_OH1_L1_3_1a' # 5 Better signal of course. Lots of L3 one moving

#Scope Parameters
# FOV = 50e-6
cam_px = 5.86e-6
totalmag = 37.5
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

1.5626666666666666e-07
0.341296928327645
12.79863481228669


### Loading center planes for evaluation

In [82]:
%%time
#Load Only center Camera to see if good FOV and signal
try:
    center_planes = calibration.lazy_dask_stack(main_folder,num_cams=4,px_depth='uint8', height=600, width =960)
    print("8bit")
except: 
    print("16bit")
    center_planes = calibration.lazy_dask_stack(main_folder,num_cams=4,px_depth='uint16', height=600, width =960)

16bit
CPU times: total: 375 ms
Wall time: 356 ms


In [83]:
napari.view_image(center_planes, name='center_planes',scale=[z_scale,1,1],multiscale=False)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 299.5, 479.5), zoom=0.646, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 500.0, 1.0), (0.0, 51.19453924914676, 12.79863481228669), (0.0, 600.0, 1.0), (0.0, 960.0, 1.0)), current_step=(250, 2, 300, 480), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'center_planes' at 0x1ddc7adcbe0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=Intera

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\jupyter_client\threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\asyncio\base_events.py", line 592, in run_until_complete
    self._check_running()
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\asyncio\base_events.py", line 554, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages

### Loading Dataset

In [57]:
%%time
#Load Only center Camera to see if good FOV and signal
try:
    stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint8', height=600, width =960)
    print("8bit")
except: 
    print("16bit")
    stack = calibration.lazy_dask_stack(main_folder,num_cams=25,px_depth='uint16', height=600, width =960)

8bit
CPU times: total: 4.45 s
Wall time: 4.45 s


In [58]:
napari.view_image(stack, name='center_planes',scale=[z_scale,1,1],multiscale=False)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 299.5, 479.5), zoom=0.34635416666666663, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 1000.0, 1.0), (0.0, 307.1672354948805, 12.79863481228669), (0.0, 600.0, 1.0), (0.0, 960.0, 1.0)), current_step=(500, 11, 300, 480), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'center_planes' at 0x1dd6b001e20>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(intera

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\jupyter_client\threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\asyncio\base_events.py", line 592, in run_until_complete
    self._check_running()
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\asyncio\base_events.py", line 554, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages

### Align Datasets with Known Offsets


In [16]:
# Load offsets from desired folder 
main_folder_offset = r'E:\Ed\PSF_defocus\20220404_M25_tamronArray_mfgideal\20220405_M25_PSF2'
offsets_file_load = os.path.join(main_folder_offset,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

In [17]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(500, 24, 600, 960)


In [18]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
da_stack_align[:,12,:,:]= da_stack_align[:,12,:,:]*0.86174

print(da_stack_align)

dask.array<setitem, shape=(500, 24, 600, 960), dtype=uint8, chunksize=(125, 24, 150, 160), chunktype=numpy.ndarray>


In [8]:
viewer= napari.Viewer()

In [20]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1], multiscale=False)

<Image layer 'dask_align' at 0x1791fc73c10>

In [18]:
# # Align the files
# ij_stack2  = ij.py.to_java(np.array(da_stack_align,dtype='uint16'))
# ij.ui().show('PSF', ij_stack2)

### CROP PSF TO ANALYZE


In [10]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

[[  1.          12.         140.70880453 134.50301597]
 [  1.          12.         140.70880453 667.62400002]
 [  1.          12.         587.47920665 667.62400002]
 [  1.          12.         587.47920665 134.50301597]]
[[141 135]
 [587 668]]
(2, 2)


In [11]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

1 12


<Image layer 'crop' at 0x1790597c670>

### Image J Part

In [12]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init('net.imagej:imageJ:2.1.0',mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

In [13]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_img))
ij.ui().show('PSF', ij_stack2)

2.1.0/1.53c


In [29]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
# anp2 = anp[:,:,:,np.newaxis]
# print(anp2.shape)
# anp2 = np.swapaxes(anp,1,4)
# anp2.shape

(500, 316, 336, 24)


In [30]:
anp2 = np.swapaxes(anp,1,3)
print(anp2.shape)
anp2_da = da.asarray(anp2)
viewer.add_image(anp2_da, name='crop',scale=[z_scale,1,1])

(500, 24, 336, 316)


<Image layer 'crop [1]' at 0x280d4ce3730>

#### Save TIFF FIles

In [45]:
import tifffile

filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)
print(filepath_save)
tifffile.imsave(filepath_save,anp2_da)

E:\Ed\Data\20220405_drosophila_egg_larvae\20220405_M25_larvae3b\crop_section.ome.tif


C:\Users\Callisto\AppData\Local\Temp\ipykernel_8980\34166168.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,anp2_da)


### Read TIFF FILE

In [4]:
viewer = napari.Viewer()

In [5]:
filename = 'crop_section' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)

In [7]:
crop_tiff = tifffile.imread(filepath_save)
print(crop_tiff.shape)
print(type(crop_tiff))
da_crop_tiff = da.from_array(crop_tiff)
viewer.add_image(crop_tiff, name='crop',scale=[z_scale,1,1],contrast_limits=(0,2**8-1))

(250, 25, 288, 288)
<class 'numpy.ndarray'>


<Image layer 'crop' at 0x19786245cd0>

C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

Rendering frames...


  0%|          | 0/61 [00:00<?, ?it/s]C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
C:\Users\Callisto\Anaconda3\envs\napari-pymmcore\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), 

c


In [8]:
from napari_animation import Animation
animation = Animation(viewer)
animation.animate('test.gif',canvas_only = False )

### Playing with napari-animation plugin


In [41]:
from napari_animation import AnimationWidget

animation_widget = AnimationWidget(viewer)
viewer.window.add_dock_widget(animation_widget, area='right')

ModuleNotFoundError: No module named 'napari_animation'